# Import Libraries

In [ ]:
import numpy as np
import os, cv2, random
import pandas as pd
import seaborn as sb
from random import shuffle 
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import sys
from typing import List
import csv
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.applications.vgg16 import VGG16
from numpy.random import permutation
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

Initialze the path to directories

In [ ]:
base_dir = '../input/ee4483/datasets'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

train_cat_dir = os.path.join(train_dir, 'cat')
train_dog_dir = os.path.join(train_dir, 'dog')

val_cat_dir = os.path.join(val_dir, 'cat')
val_dog_dir = os.path.join(val_dir, 'dog')

# Data Exploration

Class Distribution

In [ ]:
print('Number of Images of cat for training: ', len(os.listdir(train_cat_dir)))
print('Number of Images of dog for training: ', len(os.listdir(train_dog_dir)))
print('Number of Images of cat for validation: ', len(os.listdir(val_cat_dir)))
print('Number of Images of dog for validation: ', len(os.listdir(val_dog_dir)))

In [ ]:
class_names = ['cat', 'dog']
train_count = [len(os.listdir(train_cat_dir)), len(os.listdir(train_dog_dir))]
val_count = [len(os.listdir(val_cat_dir)), len(os.listdir(val_dog_dir))]

X_axis = np.arange(len(class_names))
  
plt.bar(X_axis - 0.2, train_count, 0.4, label = 'Train')
plt.bar(X_axis + 0.2, val_count, 0.4, label = 'Validation')
  
plt.xticks(X_axis, class_names)
plt.xlabel("Classes")
plt.ylabel("Number of images")
plt.title("Number of images for training and validation for each class")
plt.legend(bbox_to_anchor = (1.28, 1))
plt.show()


Dataset Distribution

In [ ]:
train = len(os.listdir(train_cat_dir))+len(os.listdir(train_dog_dir))
val = len(os.listdir(val_cat_dir))+len(os.listdir(val_dog_dir))
test = len(os.listdir(test_dir))

x = ["train", "val", "test"]
height = [train, val, test]
plt.bar(x = x, height = height)
plt.title("Dataset distribution")
plt.xlabel("Sets")
plt.ylabel("Number of images")
plt.show()

Show Images from training and validation datasets

In [ ]:
train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(224, 224),
                                             seed = 123)
validation_dataset = image_dataset_from_directory(val_dir,
                                                  shuffle=True,
                                                  batch_size=32,
                                                  image_size=(224,224),
                                                  seed = 123)
class_names = train_dataset.class_names

In [ ]:
def plot_images(dataset):
    #This function plots the images from training or validation datasets
    class_names = dataset.class_names
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")

After data loading, we explore the data by visualizing some images from the training and validation sets
by plotting some sample images and their corresponding labels

In [ ]:
plot_images(train_dataset)

In [ ]:
plot_images(validation_dataset)

# Define Baseline Model

In [ ]:
def build_baseline_CNN():
    #This function is used to build the baseline CNN model
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))

    return model

Plot learning curves

In [ ]:
def plot_result(history, k=0):
    #This function is used to plot the learning curves
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(len(acc))

    #Plot accuracy 
    plt.plot(epochs, acc, label='Training accuracy')
    plt.plot(epochs, val_acc, label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.margins(0.05, tight=True)
    plt.legend(loc='lower right')
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename +'_'+str(k)+'_acc_plot.png')
    plt.figure()

    #Plot Loss
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.margins(0.05, tight=True)
    plt.legend(loc='upper right')
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename +'_'+str(k)+'_loss_plot.png')
    plt.show()


Run Basline Model Without Augmentation 

In [ ]:
def run_baseline():
    #This function is used to train and evaluate the baseline CNN model
    
    #build the model
    model = build_baseline_CNN() 
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    # prepare iterators
    train_ds = datagen.flow_from_directory(train_dir,
                                           class_mode='binary', batch_size=32, target_size=(224, 224))
    val_ds = datagen.flow_from_directory(val_dir,
                                         class_mode='binary', batch_size=32, target_size=(224, 224))
    #compile the model
    model.compile(optimizer=Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])  
    model.summary()
    # fit the model
    history = model.fit(train_ds, steps_per_epoch=len(train_ds),
                        validation_data=val_ds, validation_steps=len(val_ds), epochs=20)
    model.save('Baseline_CNN_w/o_aug.h5')
    # evaluate the model
    score1 = model.evaluate(train_ds, verbose =0)
    score2 = model.evaluate(val_ds, verbose=0)
    print('Training Accuracy  : %1.2f%%     Training loss  : %1.6f'%(score1[1]*100,score1[0]))
    print('Validation Accuracy: %1.2f%%     Validation loss: %1.6f'%(score2[1]*100,score2[0]))
    # plot the learning curves
    plot_result(history)

In [ ]:
run_baseline()

# Data Loading and Data Augmentation

In [ ]:
def load_data():
    #This function is used to load data and perform preprocessing
    
    train_datagen = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 40, 
                                     width_shift_range = 0.2, 
                                     height_shift_range = 0.2, 
                                     shear_range = 0.2, 
                                     zoom_range = 0.2, 
                                     horizontal_flip = True)

    validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

    train_dataset = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   seed =123)
    validation_dataset = validation_datagen.flow_from_directory(val_dir,
                                                      target_size=(224, 224),
                                                      batch_size=32,
                                                      class_mode='binary',
                                                      seed =123)
    return train_dataset, validation_dataset

Visualize images after augmentation

In [ ]:
train_ds, val_ds = load_data()

In [ ]:
x= train_ds.next()
plt.figure(figsize=(10, 10))
image=x[0]
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image[i])

# Build Models

In [ ]:
def build_model_CNN():  
    #This function is used to build the CNN model with dropout regularization

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
def build_model_VGG16():
    #This function is used to build the model with a VGG-16 pretrained feature extraction backbone
    
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    
    # loaded layers set to be not trainable
    for layer in model.layers:
        layer.trainable = False
        
    # addition of classification layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    
    # define the model
    model = Model(inputs=model.inputs, outputs=output)
    
    return model

# Training for Different Parameters and Log the Results

Training Function

In [ ]:
def train_fn(model, hp:dict, train_dataset, validation_dataset, k) -> dict:
    #This function runs the training loop and tests the model on validation data
    
    checkpointer = ModelCheckpoint(filepath = "weights_best_hp_"+str(k)+".h5", save_best_only = True, save_weights_only = True)
    
    history = model.fit(train_dataset,
                          validation_data=validation_dataset,
                          epochs=hp['epoch'],
                          callbacks = [checkpointer])
  
    model.save('./VGG_hp_'+str(k)+'.h5') #change to model.save('./CNN_hp_'+str(k)+'.h5') when using CNN model
  
    #Visualize training result
    plot_result(history,k)
    return history

Log Results for each configuration

In [ ]:
def parse_results(hp, score1, score2):
    # This function logs the results of the parameter configuration 
    
    data = [hp['epoch'], hp['optimizer'], hp['lr'], score1[0], score1[1], score2[0], score2[1]]
    df = pd.Series(data).to_frame()   
    filename = './VGG_Records.csv'  #change to filename = './CNN_Records.csv' when used with CNN model
    with open(filename, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(data)

Training Engine for different parameter settings

In [ ]:
def run_experiments(exp_hp: List[dict]):
    #This function is used to run different parameter configurations and to log the results
    k = 0
    for hp in exp_hp:
        tf.random.set_seed(123)

        # Create the datasets
        train_dataset, validation_dataset = load_data()
        
        #build model
        model = build_model_VGG16() #change to model = build_model_CNN() if CNN model is to be used 
        model.summary()
        
        #Assign optimizer 
        if hp['optimizer'] == 'Adam':
            opt = Adam(learning_rate=hp['lr'])
        elif hp['optimizer'] == 'SGD':
            opt = SGD(learning_rate=hp['lr'], momentum=0.9)
        else:
            opt = RMSprop(learning_rate=hp['lr'])
            
        #compile model
        model.compile(optimizer=opt,
                      loss=tf.keras.losses.binary_crossentropy,
                      metrics=['accuracy'])
        
        print('Parameter set: ', hp)
        # Call the training function
        history = train_fn(model, hp, train_dataset, validation_dataset, k+1)
        #Evaluate the model
        score1 = model.evaluate(train_dataset, verbose =0)
        score2 = model.evaluate(validation_dataset, verbose=0)
        print('Training Accuracy  : %1.2f%%     Training loss  : %1.6f'%(score1[1]*100,score1[0]))
        print('Validation Accuracy: %1.2f%%     Validation loss: %1.6f'%(score2[1]*100,score2[0]))
        #Log Results
        parse_results(hp, score1, score2) 
        k += 1

Define different parameter configurations used with each model

In [ ]:
hps = [{
          "epoch": 10,
          "optimizer": 'Adam',
          "lr": 0.00001
       },
       {
          "epoch": 10,
          "optimizer": 'Adam',
          "lr": 0.001
       },
       {
          "epoch": 10,
          "optimizer": 'SGD',
          "lr": 0.001
       }, 
       {
          "epoch": 10,
          "optimizer": 'SGD',
          "lr": 0.01
       },
       {
          "epoch": 10,
          "optimizer": 'RMSProp',
          "lr": 0.00001
       },
       {
          "epoch": 10,
          "optimizer": 'RMSProp',
          "lr": 0.00001
       },
       {
          "epoch": 20,
          "optimizer": 'Adam',
          "lr": 0.00001
       },
       {
          "epoch": 20,
          "optimizer": 'Adam',
          "lr": 0.001
       },
       {
          "epoch": 20,
          "optimizer": 'SGD',
          "lr": 0.001
       }, 
       {
          "epoch": 20,
          "optimizer": 'SGD',
          "lr": 0.01
       },
       {
          "epoch": 20,
          "optimizer": 'RMSProp',
          "lr": 0.00001
       },
       {
          "epoch": 20,
          "optimizer": 'RMSProp',
          "lr": 0.00001
       }
]    

In [ ]:
run_experiments(hps)

# Prediction

In [ ]:
def prediction(model):
    #This function is used to obtain predictions on test data and visulaize some predictions
    
    #Prepare test data
    test_filenames = os.listdir(test_dir)
    test_df = pd.DataFrame({
      'filename': test_filenames
    })
    nb_samples = test_df.shape[0]
    test_gen = ImageDataGenerator(rescale=1./255.)
    test_generator = test_gen.flow_from_dataframe(
      test_df, 
      test_dir, 
      x_col='filename',
      y_col=None,
      class_mode=None,
      target_size=(224,224),
      batch_size=1,
      shuffle=False
    )
  
    #prediction
    predict = model.predict(test_generator, steps=np.ceil(nb_samples))
    threshold = 0.5
    test_df['category'] = np.where(predict > threshold, 1,0)

    #visualize predictions
    sample_test = test_df.head(18)
    sample_test.head()
    plt.figure(figsize=(12, 24))
    for index, row in sample_test.iterrows():
        filename = row['filename']
        category = row['category']
        img = load_img(test_dir+'/'+filename, target_size=(224,224))
        ax = plt.subplot(6, 3, index+1)
        plt.imshow(img)
        plt.title(filename + '(' + "{}".format(category) + ')' )
        plt.axis("off")

    plt.figure(figsize=(10,5))
    sb.countplot(test_df['category'])
    plt.title("(Test data)")
  
    return test_df 
  

# Loading the best model for predictions

In [ ]:
model = load_model('../input/vgg-adam-94/VGG_hp_2 (1).h5')
test_df = prediction(model)

# Submission.csv file 

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df = submission_df.astype({'id': 'int64'})
submission_df.sort_values(by = 'id', inplace=True)
submission_df.to_csv('submission.csv', index=False)

# Prediction Analysis

In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

validation_dataset = validation_datagen.flow_from_directory(val_dir,
                                                      target_size=(224, 224),
                                                      batch_size=1,
                                                      class_mode='binary',
                                                      seed =123,
                                                      shuffle=False)


In [ ]:
val_labels = validation_dataset.labels
filenames=validation_dataset.filenames
    
preds = model.predict(validation_dataset)
    
val_pred_labels = np.where(preds > 0.5, 1,0)
val_pred_labels = val_pred_labels.transpose()
    

In [ ]:
def plots(ims, figsize=(12, 6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0, 2, 3, 1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        
def plots_idx(idx, titles=None):
    plots([load_img(val_dir+'/' + filenames[i]) for i in idx], titles=titles)

Plotting the correct predictions from validation set

In [ ]:
n_view = 4
correct = np.where(val_pred_labels==val_labels)[1]
idx = permutation(correct)[:n_view]
plots_idx(idx, val_pred_labels[0][idx])

Plotting the misclassified predictions from validation set

In [ ]:
n_view = 4
incorrect = np.where(val_pred_labels!=val_labels)[1]
idx = permutation(incorrect)[:n_view]
plots_idx(idx, val_pred_labels[0][idx])

# CIFAR-10 DATASET

Data Loading

In [ ]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

Plot some training images

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(15,15))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(trainX[i])
    plt.xlabel(labels[trainY[i][0]],fontsize=12)
plt.show()

Data Preparation

In [ ]:
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)
   
# prepare pixel data
trainX = trainX.astype('float32')
testX = testX.astype('float32')
# normalize to range 0-1
trainX = trainX / 255.0
testX = testX / 255.0

Build the model for classification

In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    # compile model
    opt = 'adam'
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Plot learning curves

In [ ]:
def plot_results(history):
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(len(acc))

    #Plot accuracy 
    plt.plot(epochs, acc, label='Training accuracy')
    plt.plot(epochs, val_acc, label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.margins(0.05, tight=True)
    plt.legend(loc='lower right')
    
    plt.figure()

    #Plot Loss
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.margins(0.05, tight=True)
    
    plt.show()    

Data Augmentaion and fit the model

In [ ]:
# define model
model = define_model()

# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
# prepare iterator
train_ds = datagen.flow(trainX, trainY, batch_size=64)
# fit model
steps = int(trainX.shape[0] / 64)
history = model.fit(train_ds, steps_per_epoch=steps, epochs=80, validation_data=(testX, testY))
model.save('./Cifar-10.h5')
plot_results(history)

Evaluation and prediction

In [ ]:
scores1 = model.evaluate(testX, testY)
scores2 = model.evaluate(trainX, trainY)
print('Test loss:', scores1[0])
print('Test accuracy:', scores1[1])
print('Training loss:', scores2[0])
print('Training accuracy:', scores2[1])

# make prediction.
pred = model.predict(testX)

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None, cbar_kw={}, cbarlabel="", **kwargs):
    
    #This function is used to create a heatmap from a numpy array and two lists of labels.
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)
    
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)
    
    ax.set_xlabel('Predicted Label') 
    ax.set_ylabel('True Label')
    
    return im, cbar

def annotate_heatmap(im, data=None, fmt="d", threshold=None):
    #This function is used to annotate a heatmap.
    
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            text = im.axes.text(j, i, format(data[i, j], fmt), horizontalalignment="center",
                                 color="white" if data[i, j] > thresh else "black")
            texts.append(text)

    return texts

Heat Map

In [ ]:
Y_pred_classes = np.argmax(pred, axis=1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(testY, axis=1)
# Errors are difference between predicted labels and true labels

cm = confusion_matrix(Y_true, Y_pred_classes) 
thresh = cm.max() / 2.

fig, ax = plt.subplots(figsize=(12,12))
im, cbar = heatmap(cm, labels, labels, ax=ax,
                   cmap=plt.cm.Blues, cbarlabel="count of predictions")
texts = annotate_heatmap(im, data=cm, threshold=thresh)

fig.tight_layout()
plt.show()

Classification Report

In [ ]:
print(classification_report(Y_true, Y_pred_classes))

Visulaization of predictions

In [ ]:
R = 3
C = 5
fig, axes = plt.subplots(R, C, figsize=(12,8))
axes = axes.ravel()

for i in np.arange(0, R*C):
    axes[i].imshow(testX[i])
    axes[i].set_title("True: %s \nPredict: %s" % (labels[Y_true[i]], labels[Y_pred_classes[i]]))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)

Incorrect predictions

In [ ]:
R = 3
C = 5
fig, axes = plt.subplots(R, C, figsize=(12,8))
axes = axes.ravel()

misclassified_idx = np.where(Y_pred_classes != Y_true)[0]
for i in np.arange(0, R*C):
    axes[i].imshow(testX[misclassified_idx[i]])
    axes[i].set_title("True: %s \nPredicted: %s" % (labels[Y_true[misclassified_idx[i]]], 
                                                  labels[Y_pred_classes[misclassified_idx[i]]]))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)